<a href="https://colab.research.google.com/github/Riddars/ITMO_Project_Cytotoxic/blob/%D0%94%D0%BE%D0%B1%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B4%D0%B5%D1%81%D0%BA%D1%80%D0%B8%D0%BF%D1%82%D0%BE%D1%80%D0%BE%D0%B2/All_Descriptors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pubchempy
!pip install requests
!pip install rdkit-pypi
import pandas as pd
from pubchempy import get_compounds
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors

In [ ]:
DataFrame_CID = pd.read_csv('/content/3.Cytotoxicity_(add SMILES).csv', encoding='ISO-8859-1')


In [ ]:
DataFrame_CID.head()

,Cell type,test,material,time (hr),concentration (ug/ml),viability (%),Hydrodynamic diameter (nm),Zeta potential (mV),CID,Canonical Smiles,...,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi,Canonical Smiles
0,143B,MTT,Au,4,200.0,36.126,24.6,-4.65,23985,[Au],...,0.0,0.0,0.0,0.0,0.74026,1.74026,0.091137,2.143769,0.158602,[Au]
1,143B,MTT,Au,4,400.0,22.253,24.6,-33.33,23985,[Au],...,0.0,0.0,0.0,0.0,0.74026,1.74026,0.091137,2.143769,0.158602,[Au]
2,143B,MTT,Au,4,500.0,19.643,24.6,-31.30,23985,[Au],...,0.0,0.0,0.0,0.0,0.74026,1.74026,0.091137,2.143769,0.158602,[Au]
3,143B,MTT,Au,72,1.0,91.262,24.6,-16.95,23985,[Au],...,0.0,0.0,0.0,0.0,0.74026,1.74026,0.091137,2.143769,0.158602,[Au]
4,143B,MTT,Au,72,3.0,87.500,24.6,29.45,23985,[Au],...,0.0,0.0,0.0,0.0,0.74026,1.74026,0.091137,2.143769,0.158602,[Au]


In [ ]:
descriptor_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
get_descriptors = rdMolDescriptors.Properties(descriptor_names)
num_descriptors = len(descriptor_names)

# Инициализация пустой матрицы для дескрипторов
descriptors_set = np.empty((0, num_descriptors), float)
smiles_list = []

# Замените DataFrame_CID на вашу переменную
for _, row in DataFrame_CID.iterrows():
    smiles = row['Canonical Smiles']
    molecule = Chem.MolFromSmiles(smiles)

    if molecule is not None:
        descriptors = np.array(get_descriptors.ComputeProperties(molecule)).reshape((-1, num_descriptors))
        descriptors_set = np.append(descriptors_set, descriptors, axis=0)
        smiles_list.append(smiles)

# Создание DataFrame с дескрипторами и SMILES
df_descriptors = pd.DataFrame(descriptors_set, columns=descriptor_names)
df_descriptors['Canonical Smiles'] = smiles_list

# Объединение существующего датасета с новыми дескрипторами
DataFrame_CID = pd.concat([DataFrame_CID, df_descriptors], axis=1)

In [ ]:
DataFrame_2 = pd.read_csv('/content/cell_line_descriptors.csv', encoding='ISO-8859-1')

In [ ]:
# Выполните объединение по столбцам "Cell type" и "cell line"
merged_df = DataFrame_CID.merge(DataFrame_2, left_on="Cell type", right_on="cell line", how="inner")

In [ ]:
merged_df.head()

,Cell type,test,material,time (hr),concentration (ug/ml),viability (%),Hydrodynamic diameter (nm),Zeta potential (mV),CID,Canonical Smiles,...,kappa3,Phi,Canonical Smiles,cell line,organism,cell type,morphology,tissue,disease,BSL
0,143B,MTT,Au,4,200.0,36.126,24.6,-4.65,23985,[Au],...,2.143769,0.158602,[Au],143B,Human,Human,mix,bone,osteosarcoma,1
1,143B,MTT,Au,4,400.0,22.253,24.6,-33.33,23985,[Au],...,2.143769,0.158602,[Au],143B,Human,Human,mix,bone,osteosarcoma,1
2,143B,MTT,Au,4,500.0,19.643,24.6,-31.30,23985,[Au],...,2.143769,0.158602,[Au],143B,Human,Human,mix,bone,osteosarcoma,1
3,143B,MTT,Au,72,1.0,91.262,24.6,-16.95,23985,[Au],...,2.143769,0.158602,[Au],143B,Human,Human,mix,bone,osteosarcoma,1
4,143B,MTT,Au,72,3.0,87.500,24.6,29.45,23985,[Au],...,2.143769,0.158602,[Au],143B,Human,Human,mix,bone,osteosarcoma,1


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2964 entries, 0 to 2963
Data columns (total 61 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Cell type                        2964 non-null   object 
 1   test                             2964 non-null   object 
 2   material                         2964 non-null   object 
 3   time (hr)                        2964 non-null   int64  
 4   concentration (ug/ml)            2964 non-null   float64
 5   viability (%)                    2964 non-null   float64
 6   Hydrodynamic diameter (nm)       2964 non-null   float64
 7   Zeta potential (mV)              2964 non-null   float64
 8   CID                              2964 non-null   int64  
 9   Canonical Smiles                 2964 non-null   object 
 10  exactmw                          2964 non-null   float64
 11  amw                              2964 non-null   float64
 12  lipinskiHBA         

In [ ]:
merged_df.to_csv("merged_df.csv", index=False)